In [4]:
import math
import pandas as pd
from dotenv import load_dotenv
import os

In [5]:
import requests
from urllib.parse import urlencode
import base64
import webbrowser

In [6]:
load_dotenv()

True

In [7]:
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

In [8]:
# Define the endpoint for obtaining access tokens
token_url = 'https://accounts.spotify.com/api/token'

# Define the payload for the request
payload = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Make the request to obtain the access token
response = requests.post(token_url, data=payload)

# Check if the request was successful
if response.status_code == 200:
    # Extract the access token from the response
    access_token = response.json()['access_token']
    print("Access Token:", access_token)
else:
    print("Error:", response.status_code, response.text)

Access Token: BQDcwYWE1oj7kJnBZc6intgOjB3LxLREIOLY5saXOrylZfeqS3wyX8XfYZOdHva3jZE5MzkkloADA9f11WNClR9ph8i-ePlvNlHcHBVZxnv7kiBon9M


In [9]:
artist_id = '3WrFJ7ztbogyGnTHbHJFl2'  # The Beatles

# Define the endpoint for fetching albums by the artist
albums_url = f'https://api.spotify.com/v1/artists/{artist_id}/albums'

# Define the headers with the access token
headers = {
    'Authorization': f'Bearer {access_token}'
}

In [10]:
# Initialize lists to store track details
track_names = []
album_names = []
tempos = []
keys = []
modes = []
release_dates = []
energies = []
danceabilities = []
loudnesses = []
speechinesses = []
acousticnesses = []
instrumentalnesses = []
livenesses = []
time_signatures = []
valences = []
durations = []


In [11]:
# Make the request to fetch albums by the artist
response = requests.get(albums_url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Extract album details
    albums_data = response.json()['items']
    
    # Iterate over each album
    for album in albums_data:
        album_name = album['name']
        
        # Fetch tracks of the album
        album_tracks_url = album['href']
        album_response = requests.get(album_tracks_url, headers=headers)
        
        # Check if the request for album tracks was successful
        if album_response.status_code == 200:
            # Extract track details
            tracks_data = album_response.json()['tracks']['items']
            
            # Iterate over each track
            for track in tracks_data:
                track_names.append(track['name'])
                album_names.append(album_name)
                
                # Fetch audio features of the track
                track_id = track['id']
                audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
                audio_features_response = requests.get(audio_features_url, headers=headers)
                
                # Check if the request for audio features was successful
                if audio_features_response.status_code == 200:
                    # Extract audio features
                    audio_features = audio_features_response.json()
                    tempos.append(audio_features['tempo'])
                    keys.append(audio_features['key'])
                    modes.append(audio_features['mode'])
                    energies.append(audio_features['energy'])
                    danceabilities.append(audio_features['danceability'])
                    loudnesses.append(audio_features['loudness'])
                    speechinesses.append(audio_features['speechiness'])
                    acousticnesses.append(audio_features['acousticness'])
                    instrumentalnesses.append(audio_features['instrumentalness'])
                    livenesses.append(audio_features['liveness'])
                    time_signatures.append(audio_features['time_signature'])
                    valences.append(audio_features['valence'])
                    durations.append(audio_features['duration_ms'])
                    
                else:
                    tempos.append(None)
                    keys.append(None)
                    modes.append(None)
                    energies.append(None)
                    danceabilities.append(None)
                    loudnesses.append(None)
                    speechinesses.append(None)
                    acousticnesses.append(None)
                    instrumentalnesses.append(None)
                    livenesses.append(None)
                    time_signatures.append(None)
                    valences.append(None)
                    durations.append(None)
                
                # Fetch release date of the track
                track_info_url = f'https://api.spotify.com/v1/tracks/{track_id}'
                track_info_response = requests.get(track_info_url, headers=headers)
                
                # Check if the request for track information was successful
                if track_info_response.status_code == 200:
                    # Extract release date
                    release_date = track_info_response.json()['album']['release_date']
                    release_dates.append(release_date)
                else:
                    release_dates.append(None)
                
        else:
            print("Error fetching tracks for album:", album['name'])

else:
    print("Error fetching albums:", response.status_code, response.text)

# Create a DataFrame
df = pd.DataFrame({
    'Track Name': track_names,
    'Album Name': album_names,
    'Tempo': tempos,
    'Key': keys,
    'Mode': modes,
    'Release Date': release_dates,
    'Energy': energies,
    'Danceability': danceabilities,
    'Loudness': loudnesses,
    'Speechiness': speechinesses,
    'Acousticness': acousticnesses,
    'Instrumentalness': instrumentalnesses,
    'Liveness': livenesses,
    'Time Signature': time_signatures,
    'Valence': valences,
    'Duration': durations
})

# Display the DataFrame
df.head()


,Track Name,Album Name,Tempo,Key,Mode,Release Date,Energy,Danceability,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Time Signature,Valence,Duration
0,Strawberry Fields Forever - 2015 Mix,The Beatles 1967 – 1970 (2023 Edition),96.221,10,1,2023-11-10,0.678,0.391,-8.773,0.3060,0.2730,0.000257,0.0965,4,0.245,252093
1,Penny Lane - 2017 Mix,The Beatles 1967 – 1970 (2023 Edition),113.225,9,1,2023-11-10,0.505,0.647,-8.260,0.0351,0.0715,0.002810,0.1840,4,0.682,180947
2,Sgt. Pepper's Lonely Hearts Club Band - 2017 Mix,The Beatles 1967 – 1970 (2023 Edition),95.686,7,1,2023-11-10,0.825,0.368,-7.758,0.0997,0.0308,0.020000,0.9080,4,0.524,122120
3,With A Little Help From My Friends - 2017 Mix,The Beatles 1967 – 1970 (2023 Edition),112.240,4,1,2023-11-10,0.586,0.673,-7.946,0.0302,0.1330,0.000000,0.2520,4,0.732,164373
4,Lucy In The Sky With Diamonds - 2017 Mix,The Beatles 1967 – 1970 (2023 Edition),187.861,11,0,2023-11-10,0.539,0.244,-6.984,0.0457,0.1540,0.000000,0.1820,4,0.429,210453


In [12]:
df.to_csv('spotify_data.csv', index=False)

In [15]:
import plotly.express as px
df['Release Date'] = pd.to_datetime(df['Release Date'])
df = df.sort_values(by='Release Date')


In [16]:
fig = px.scatter(df, x='Release Date', y='Tempo', color='Key', hover_name='Track Name', 
                 title='Time Series of Tempo and Key based on Release Date', 
                 labels={'Tempo': 'Tempo (BPM)', 'Release Date': 'Release Date', 'Key': 'Key'})

# Customize the layout
fig.update_traces(marker=dict(size=8, opacity=0.7),
                  selector=dict(mode='markers'))

# Show the plot
fig.show()

In [17]:
df2 = df[df['Release Date'].dt.year < 1975]

In [18]:
fig = px.scatter(df2, x='Release Date', y='Tempo', color='Key', hover_name='Track Name', 
                 title='Time Series of Tempo and Key based on Release Date', 
                 labels={'Tempo': 'Tempo (BPM)', 'Release Date': 'Release Date', 'Key': 'Key'})

# Customize the layout
fig.update_traces(marker=dict(size=8, opacity=0.7),
                  selector=dict(mode='markers'))

# Show the plot
fig.show()